## Checking out the other data sources

In [9]:
import _report_utils
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import rt_dates, rt_utils

alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)
from typing import List, Union

In [10]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [11]:
name = "SBMTD Schedule"

In [12]:
date = "2024-03-13"

### route_direction_metrics

In [13]:
vp_route_direction_metrics =  pd.read_parquet(
    f"{RT_SCHED_GCS}vp_route_dir/route_direction_metrics_{date}.parquet",
    filters=[[("name", "==", name)]])

In [14]:
vp_route_direction_metrics.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,name,base64_url,organization_source_record_id,organization_name,caltrans_district
696,239f3baf3dd3b9e9464f66a777f9897d,1,0.00,offpeak,448,117,469.75,390.00,571,524,0,17,9,26,1.22,0.92,0.95,0.25,1.00,0.30,1.20,18.07,SBMTD Schedule,aHR0cHM6Ly9zYm10ZC5nb3YvZ29vZ2xlX3RyYW5zaXQvZmVlZC56aXA=,recswCrw6a6htmXJ4,Santa Barbara Metropolitan Transit District,05 - San Luis Obispo
697,239f3baf3dd3b9e9464f66a777f9897d,1,0.00,peak,496,134,515.92,405.00,637,589,0,17,10,27,1.23,0.93,0.96,0.26,1.00,0.33,1.27,19.11,SBMTD Schedule,aHR0cHM6Ly9zYm10ZC5nb3YvZ29vZ2xlX3RyYW5zaXQvZmVlZC56aXA=,recswCrw6a6htmXJ4,Santa Barbara Metropolitan Transit District,05 - San Luis Obispo
